In [4]:
import sys
sys.path.insert(1, '/home/nicolas/Stage/code/stage/src')
sys.path.insert(1, '/home/nicolas/Stage/code/stage/data')
sys.path.insert(1, '/home/nicolas/Stage/code/stage/model')

from torch.optim.lr_scheduler import StepLR
from ComparisonDataset import ComparisonDataset
from BinaryClassifier import BinaryClassifier, train, test
import torch.optim as optim
import torch
import functions
import numpy as np
import gzip
import pickle
device = torch.device("cuda")
dtype = torch.float

fname = "AllParameters_LongRUNExMC_nMB500_NS50000_TEMP_0_MNIST_Nh500_lr0.01_l20.0_NGibbs10000.h5"


In [5]:
f = gzip.open('./data/mnist.pkl.gz', 'rb')
u = pickle._Unpickler(f)
u.encoding = 'latin1'
p = u.load()
train_set, _, _ = p

data_mnist = torch.as_tensor(
    train_set[0][:10000, :].T, device=device, dtype=dtype)
labels_mnist = torch.tensor(np.zeros(len(data_mnist[0])), device=device)

train_kwargs = {'batch_size': 128}
test_kwargs = {'batch_size': 1000}
cuda_kwargs = {'num_workers': 0,
               'pin_memory': False,
               'shuffle': False}
train_kwargs.update(cuda_kwargs)
test_kwargs.update(cuda_kwargs)




In [8]:
fname = "model/" + fname
myRBM, f, alltimes = functions.retrieveRBM(device, fname)

times = np.array(alltimes)[range(len(alltimes))]
tmp = []
for t in [times[-1]]:
    myRBM.W = torch.tensor(f['paramW'+str(t)], device=myRBM.device)
    myRBM.vbias = torch.tensor(
        f['paramVB'+str(t)], device=myRBM.device)
    myRBM.hbias = torch.tensor(
        f['paramHB'+str(t)], device=myRBM.device)
    vinit = torch.bernoulli(torch.rand(
        (myRBM.Nv, 10000), device=myRBM.device, dtype=myRBM.dtype))

    si, _, _, _ = myRBM.Sampling(vinit, it_mcmc=10000)
    data_gen = torch.tensor(si, device=device)
    labels_gen = torch.tensor(np.ones(len(data_gen[0])), device=device)
    data = torch.cat((data_gen, data_mnist), dim=1)
    labels = torch.cat((labels_gen, labels_mnist), dim=0)
    data = data.T
    shuffle_index = torch.randperm(data.shape[0])
    newdata = torch.empty(data.shape)
    newlabel = torch.empty(labels.shape)
    for i in range(shuffle_index.shape[0]):
        newdata[i] = data[shuffle_index[i]]
        newlabel[i] = labels[shuffle_index[i]]

    net = BinaryClassifier()
    net.cuda()
    train_set = ComparisonDataset(
        device, data=newdata, labels=newlabel, train=True)
    test_set = ComparisonDataset(
        device, data=newdata, labels=newlabel, train=False)
    train_loader = torch.utils.data.DataLoader(
        train_set, **train_kwargs)
    test_loader = torch.utils.data.DataLoader(test_set, **test_kwargs)
    optimizer = optim.Adadelta(net.parameters(), lr=0.1)

    scheduler = StepLR(optimizer, step_size=1, gamma=0.7)
    for epoch in range(1, 100 + 1):
        train(net, device, train_loader, optimizer, epoch)
        test(net, device, test_loader)
        scheduler.step()
        tmp.append(torch.linalg.norm(net(test_set.data[:2500].cuda()).round().view(
            2500)-test_set.label[:2500].cuda(), 1).item()*100/2500)
    success_rate.append(tmp)
        

<ipython-input-8-67b609f1dce8>:16: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  data_gen = torch.tensor(si, device=device)
/home/nicolas/anaconda3/lib/python3.8/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /opt/conda/conda-bld/pytorch_1622099212932/work/c10/core/TensorImpl.h:1260.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


NameError: name 'success_rate' is not defined

In [9]:
tmp

[46.8,
 46.48,
 46.64,
 46.52,
 46.44,
 46.56,
 46.48,
 45.88,
 45.84,
 45.72,
 45.92,
 45.76,
 45.84,
 45.88,
 45.72,
 45.76,
 45.76,
 45.76,
 45.72,
 45.68,
 45.68,
 45.68,
 45.68,
 45.68,
 45.68,
 45.68,
 45.68,
 45.68,
 45.68,
 45.68,
 45.68,
 45.68,
 45.68,
 45.68,
 45.68,
 45.68,
 45.68,
 45.68,
 45.68,
 45.68,
 45.68,
 45.68,
 45.68,
 45.68,
 45.68,
 45.68,
 45.68,
 45.68,
 45.68,
 45.68,
 45.68,
 45.68,
 45.68,
 45.68,
 45.68,
 45.68,
 45.68,
 45.68,
 45.68,
 45.68,
 45.68,
 45.68,
 45.68,
 45.68,
 45.68,
 45.68,
 45.68,
 45.68,
 45.68,
 45.68,
 45.68,
 45.68,
 45.68,
 45.68,
 45.68,
 45.68,
 45.68,
 45.68,
 45.68,
 45.68,
 45.68,
 45.68,
 45.68,
 45.68,
 45.68,
 45.68,
 45.68,
 45.68,
 45.68,
 45.68,
 45.68,
 45.68,
 45.68,
 45.68,
 45.68,
 45.68,
 45.68,
 45.68,
 45.68,
 45.68]

In [10]:
times[-1]

92998